<a href="https://colab.research.google.com/github/Worachet-Ch/project2022/blob/main/random_nan_and_imputation_with_SMM_variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd #นำเข้า pandas แทนที่ด้วย pd
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as mno
import missingno as msno
from sklearn import linear_model
%matplotlib inline
import matplotlib
import sklearn 
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import time

In [2]:
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project' #เข้า drive
data_file_path = os.path.join(path,'data_notmissing.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data = pd.read_csv(data_file_path) #อ่านไฟล์ csv

Mounted at /content/drive


In [3]:
data.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)

<ipython-input-3-4cc622f169ae>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)


In [4]:
data

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


# 10 % random nan 

In [5]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'SMM10_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*10/100)) 
  index_nan = np.random.choice(globals()[f'SMM10_{i}']['SMM'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'SMM10_{i}']['SMM'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.23204636573791504 วินาที
เวลาที่ใช้ในการประมวลผล 0.0038674394289652508 นาที


## Imputation using Mean

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
t_start = time.time()
mean10_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean10_{i}'] = mean10_imputer.fit_transform(globals()[f'SMM10_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN10_{i}'] = mean_squared_error(data, globals()[f'imputed_mean10_{i}']) # for loop mean_squared_error
t_end = time.time()
t10_MEAN= (t_end- t_start)

In [8]:
MSE_MEAN10_1

0.04836159843220892

In [9]:
t_start = time.time()
mse_mean10_total = 0 
MSE_MEAN10_1001 = 0
for i in range(0,1001):
  mse_mean10_total = mse_mean10_total + globals()[f'MSE_MEAN10_{i+1}']
AMSE_SMM10_mean = mse_mean10_total/1000
print(f'AMSE_mean = {AMSE_SMM10_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mean = 0.07369633169499037
เวลาที่ใช้ในการประมวลผล 0.0010106563568115234 วินาที
เวลาที่ใช้ในการประมวลผล 1.6844272613525392e-05 นาที


## Imputation using the miss forest

In [10]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 2.7 MB/s eta 0:00:00


In [ ]:
t_start = time.time()
MF10_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf10_{i}'] = MF10_imputer.fit_transform(globals()[f'SMM10_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF10_{i}'] = mean_squared_error(data, globals()[f'imputed_mf10_{i}']) # for loop mean_squared_error
t_end = time.time()
t10_MF= (t_end- t_start)

In [12]:
t_start = time.time()
mse_mf10_total = 0 
MSE_MF10_1001 = 0
for i in range(0,1001):
  mse_mf10_total = mse_mf10_total + globals()[f'MSE_MF10_{i+1}']
AMSE_SMM10_mf = mse_mf10_total/1000
print(f'AMSE_mf = {AMSE_SMM10_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mf = 0.008377832634238292
เวลาที่ใช้ในการประมวลผล 0.001422882080078125 วินาที
เวลาที่ใช้ในการประมวลผล 2.3714701334635417e-05 นาที


## Imputation using the KNN

In [13]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [14]:
t_start = time.time()
KNN10_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN10_{i}'] = KNN10_imputer.fit_transform(globals()[f'SMM10_{i}'].copy())
  globals()[f'MSE_KNN10_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN10_{i}'])
t_end = time.time()
t10_KNN= (t_end- t_start)

In [15]:
t_start = time.time()
mse_knn10_total = 0 
MSE_KNN10_1001 = 0
for i in range(0,1001):
  mse_knn10_total = mse_knn10_total + globals()[f'MSE_KNN10_{i+1}']
AMSE_SMM10_knn = mse_knn10_total/1000
print(f'AMSE_knn = {AMSE_SMM10_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_knn = 0.05025529762583111
เวลาที่ใช้ในการประมวลผล 0.0016808509826660156 วินาที
เวลาที่ใช้ในการประมวลผล 2.8014183044433594e-05 นาที


## Imputation using the MICE

In [16]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr10_{i}'] = imp.fit_transform(globals()[f'SMM10_{i}'].copy())
  globals()[f'MSE_lr10_{i}'] = mean_squared_error(data, globals()[f'imputed_lr10_{i}'])
t_end = time.time()
t10_LR= (t_end- t_start)

In [18]:
t_start = time.time()
mse_lr10_total = 0 
MSE_lr10_1001 = 0
for i in range(0,1001):
  mse_lr10_total = mse_lr10_total + globals()[f'MSE_lr10_{i+1}']
AMSE_SMM10_lr = mse_lr10_total/1000
print(f'AMSE_lr = {AMSE_SMM10_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_lr = 0.006639505262347299
เวลาที่ใช้ในการประมวลผล 0.0010499954223632812 วินาที
เวลาที่ใช้ในการประมวลผล 1.749992370605469e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm10_{i}'] = svm_svr.fit_transform(globals()[f'SMM10_{i}'].copy())
  globals()[f'MSE_svm10_{i}'] = mean_squared_error(data, globals()[f'imputed_svm10_{i}'])
t_end = time.time()
t10_SVM= (t_end- t_start)

In [20]:
t_start = time.time()
mse_svm10_total = 0 
MSE_svm10_1001 = 0
for i in range(0,1001):
  mse_svm10_total = mse_svm10_total + globals()[f'MSE_svm10_{i+1}']
AMSE_SMM10_svm = mse_svm10_total/1000
print(f'AMSE_svm = {AMSE_SMM10_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_svm = 0.05906676756185311
เวลาที่ใช้ในการประมวลผล 0.001085042953491211 วินาที
เวลาที่ใช้ในการประมวลผล 1.8084049224853514e-05 นาที


## Imputation using the EM

### impy.em

In [21]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy10_{i}'] = impy.em(globals()[f'SMM10_{i}'].values.copy())
  globals()[f'MSE_impy10_{i}'] = mean_squared_error(data, globals()[f'imputed_impy10_{i}'])
t_end = time.time()
t10_EM= (t_end- t_start)

In [23]:
t_start = time.time()
mse_impy10_total = 0 
MSE_impy10_1001 = 0
for i in range(0,1001):
  mse_impy10_total = mse_impy10_total + globals()[f'MSE_impy10_{i+1}']
AMSE_SMM10_impy = mse_impy10_total/1000
print(f'AMSE_impy = {AMSE_SMM10_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_impy = 0.1377144934618607
เวลาที่ใช้ในการประมวลผล 0.00077056884765625 วินาที
เวลาที่ใช้ในการประมวลผล 1.2842814127604167e-05 นาที


# 20 % random nan

In [24]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'SMM20_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*20/100)) 
  index_nan = np.random.choice(globals()[f'SMM20_{i}']['SMM'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'SMM20_{i}']['SMM'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.15619158744812012 วินาที
เวลาที่ใช้ในการประมวลผล 0.0026031931241353354 นาที


## Imputation using Mean

In [25]:
from sklearn.impute import SimpleImputer

In [26]:
t_start = time.time()
mean20_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean20_{i}'] = mean20_imputer.fit_transform(globals()[f'SMM20_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN20_{i}'] = mean_squared_error(data, globals()[f'imputed_mean20_{i}']) # for loop mean_squared_error
t_end = time.time()
t20_MEAN = (t_end- t_start)

In [27]:
t_start = time.time()
mse_mean20_total = 0 
MSE_MEAN20_1001 = 0
for i in range(0,1001):
  mse_mean20_total = mse_mean20_total + globals()[f'MSE_MEAN20_{i+1}']
AMSE_SMM20_mean = mse_mean20_total/1000
print(f'AMSE_mean = {AMSE_SMM20_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mean = 0.14571530064821137
เวลาที่ใช้ในการประมวลผล 0.0012383460998535156 วินาที
เวลาที่ใช้ในการประมวลผล 2.063910166422526e-05 นาที


## Imputation using the miss forest

In [28]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF20_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf20_{i}'] = MF20_imputer.fit_transform(globals()[f'SMM20_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF20_{i}'] = mean_squared_error(data, globals()[f'imputed_mf20_{i}']) # for loop mean_squared_error
t_end = time.time()
t20_MF = (t_end- t_start)

In [30]:
t_start = time.time()
mse_mf20_total = 0 
MSE_MF20_1001 = 0
for i in range(0,1001):
  mse_mf20_total = mse_mf20_total + globals()[f'MSE_MF20_{i+1}']
AMSE_SMM20_mf = mse_mf20_total/1000
print(f'AMSE_mf = {AMSE_SMM20_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mf = 0.01692612093237803
เวลาที่ใช้ในการประมวลผล 0.0011103153228759766 วินาที
เวลาที่ใช้ในการประมวลผล 1.8505255381266274e-05 นาที


## Imputation using the KNN

In [31]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [32]:
t_start = time.time()
KNN20_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN20_{i}'] = KNN20_imputer.fit_transform(globals()[f'SMM20_{i}'].copy())
  globals()[f'MSE_KNN20_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN20_{i}'])
t_end = time.time()
t20_KNN = (t_end- t_start)

In [33]:
t_start = time.time()
mse_knn20_total = 0 
MSE_KNN20_1001 = 0
for i in range(0,1001):
  mse_knn20_total = mse_knn20_total + globals()[f'MSE_KNN20_{i+1}']
AMSE_SMM20_knn = mse_knn20_total/1000
print(f'AMSE_knn = {AMSE_SMM20_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_knn = 0.10197780894922087
เวลาที่ใช้ในการประมวลผล 0.0009043216705322266 วินาที
เวลาที่ใช้ในการประมวลผล 1.5072027842203776e-05 นาที


## Imputation using the MICE

In [34]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr20_{i}'] = imp.fit_transform(globals()[f'SMM20_{i}'].copy())
  globals()[f'MSE_lr20_{i}'] = mean_squared_error(data, globals()[f'imputed_lr20_{i}'])
t_end = time.time()
t20_LR = (t_end- t_start)

In [36]:
t_start = time.time()
mse_lr20_total = 0 
MSE_lr20_1001 = 0
for i in range(0,1001):
  mse_lr20_total = mse_lr20_total + globals()[f'MSE_lr20_{i+1}']
AMSE_SMM20_lr = mse_lr20_total/1000
print(f'AMSE_lr = {AMSE_SMM20_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_lr = 0.016610068641395866
เวลาที่ใช้ในการประมวลผล 0.0013065338134765625 วินาที
เวลาที่ใช้ในการประมวลผล 2.1775563557942707e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm20_{i}'] = svm_svr.fit_transform(globals()[f'SMM20_{i}'].copy())
  globals()[f'MSE_svm20_{i}'] = mean_squared_error(data, globals()[f'imputed_svm20_{i}'])
t_end = time.time()
t20_SVM = (t_end- t_start)

In [38]:
t_start = time.time()
mse_svm20_total = 0 
MSE_svm20_1001 = 0
for i in range(0,1001):
  mse_svm20_total = mse_svm20_total + globals()[f'MSE_svm20_{i+1}']
AMSE_SMM20_svm = mse_svm20_total/1000
print(f'AMSE_svm = {AMSE_SMM20_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_svm = 0.11956727719361529
เวลาที่ใช้ในการประมวลผล 0.0010731220245361328 วินาที
เวลาที่ใช้ในการประมวลผล 1.7885367075602213e-05 นาที


## Imputation using the EM

### impy.em

In [39]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy20_{i}'] = impy.em(globals()[f'SMM20_{i}'].values.copy())
  globals()[f'MSE_impy20_{i}'] = mean_squared_error(data, globals()[f'imputed_impy20_{i}'])
t_end = time.time()
t20_EM = (t_end- t_start)

In [41]:
t_start = time.time()
mse_impy20_total = 0 
MSE_impy20_1001 = 0
for i in range(0,1001):
  mse_impy20_total = mse_impy20_total + globals()[f'MSE_impy20_{i+1}']
AMSE_SMM20_impy = mse_impy20_total/1000
print(f'AMSE_impy = {AMSE_SMM20_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_impy = 0.27005150179447357
เวลาที่ใช้ในการประมวลผล 0.0022656917572021484 วินาที
เวลาที่ใช้ในการประมวลผล 3.776152928670247e-05 นาที


# 30 % random nan

In [42]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'SMM30_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*30/100)) 
  index_nan = np.random.choice(globals()[f'SMM30_{i}']['SMM'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'SMM30_{i}']['SMM'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.18594813346862793 วินาที
เวลาที่ใช้ในการประมวลผล 0.0030991355578104656 นาที


## Imputation using Mean

In [43]:
from sklearn.impute import SimpleImputer

In [44]:
t_start = time.time()
mean30_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean30_{i}'] = mean30_imputer.fit_transform(globals()[f'SMM30_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN30_{i}'] = mean_squared_error(data, globals()[f'imputed_mean30_{i}']) # for loop mean_squared_error
t_end = time.time()
t30_MEAN = (t_end- t_start)

In [45]:
t_start = time.time()
mse_mean30_total = 0 
MSE_MEAN30_1001 = 0
for i in range(0,1001):
  mse_mean30_total = mse_mean30_total + globals()[f'MSE_MEAN30_{i+1}']
AMSE_SMM30_mean = mse_mean30_total/1000
print(f'AMSE_mean = {AMSE_SMM30_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mean = 0.21810043574148563
เวลาที่ใช้ในการประมวลผล 0.0011782646179199219 วินาที
เวลาที่ใช้ในการประมวลผล 1.9637743631998696e-05 นาที


## Imputation using the miss forest

In [46]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF30_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf30_{i}'] = MF30_imputer.fit_transform(globals()[f'SMM30_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF30_{i}'] = mean_squared_error(data, globals()[f'imputed_mf30_{i}']) # for loop mean_squared_error
t_end = time.time()
t30_MF = (t_end- t_start)

In [48]:
t_start = time.time()
mse_mf30_total = 0 
MSE_MF30_1001 = 0
for i in range(0,1001):
  mse_mf30_total = mse_mf30_total + globals()[f'MSE_MF30_{i+1}']
AMSE_SMM30_mf = mse_mf30_total/1000
print(f'AMSE_mf = {AMSE_SMM30_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mf = 0.026320903798893902
เวลาที่ใช้ในการประมวลผล 0.0012218952178955078 วินาที
เวลาที่ใช้ในการประมวลผล 2.0364920298258465e-05 นาที


## Imputation using the KNN

In [49]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [50]:
t_start = time.time()
KNN30_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN30_{i}'] = KNN30_imputer.fit_transform(globals()[f'SMM30_{i}'].copy())
  globals()[f'MSE_KNN30_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN30_{i}'])
t_end = time.time()
t30_KNN = (t_end- t_start)

In [51]:
t_start = time.time()
mse_knn30_total = 0 
MSE_KNN30_1001 = 0
for i in range(0,1001):
  mse_knn30_total = mse_knn30_total + globals()[f'MSE_KNN30_{i+1}']
AMSE_SMM30_knn = mse_knn30_total/1000
print(f'AMSE_knn = {AMSE_SMM30_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_knn = 0.1558937612982516
เวลาที่ใช้ในการประมวลผล 0.0009582042694091797 วินาที
เวลาที่ใช้ในการประมวลผล 1.597007115681966e-05 นาที


## Imputation using the MICE

In [52]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr30_{i}'] = imp.fit_transform(globals()[f'SMM30_{i}'].copy())
  globals()[f'MSE_lr30_{i}'] = mean_squared_error(data, globals()[f'imputed_lr30_{i}'])
t_end = time.time()
t30_LR = (t_end- t_start)

In [54]:
t_start = time.time()
mse_lr30_total = 0 
MSE_lr30_1001 = 0
for i in range(0,1001):
  mse_lr30_total = mse_lr30_total + globals()[f'MSE_lr30_{i+1}']
AMSE_SMM30_lr = mse_lr30_total/1000
print(f'AMSE_lr = {AMSE_SMM30_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_lr = 0.03335250801487467
เวลาที่ใช้ในการประมวลผล 0.0014984607696533203 วินาที
เวลาที่ใช้ในการประมวลผล 2.4974346160888672e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm30_{i}'] = svm_svr.fit_transform(globals()[f'SMM30_{i}'].copy())
  globals()[f'MSE_svm30_{i}'] = mean_squared_error(data, globals()[f'imputed_svm30_{i}'])
t_end = time.time()
t30_SVM = (t_end- t_start)

In [56]:
t_start = time.time()
mse_svm30_total = 0 
MSE_svm30_1001 = 0
for i in range(0,1001):
  mse_svm30_total = mse_svm30_total + globals()[f'MSE_svm30_{i+1}']
AMSE_SMM30_svm = mse_svm30_total/1000
print(f'AMSE_svm = {AMSE_SMM30_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_svm = 0.18367434726376414
เวลาที่ใช้ในการประมวลผล 0.0008852481842041016 วินาที
เวลาที่ใช้ในการประมวลผล 1.4754136403401693e-05 นาที


## Imputation using the EM

### impy.em

In [57]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy30_{i}'] = impy.em(globals()[f'SMM30_{i}'].values.copy())
  globals()[f'MSE_impy30_{i}'] = mean_squared_error(data, globals()[f'imputed_impy30_{i}'])
t_end = time.time()
t30_EM = (t_end- t_start)

In [59]:
t_start = time.time()
mse_impy30_total = 0 
MSE_impy30_1001 = 0
for i in range(0,1001):
  mse_impy30_total = mse_impy30_total + globals()[f'MSE_impy30_{i+1}']
AMSE_SMM30_impy = mse_impy30_total/1000
print(f'AMSE_impy = {AMSE_SMM30_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_impy = 0.41116105057559116
เวลาที่ใช้ในการประมวลผล 0.0014767646789550781 วินาที
เวลาที่ใช้ในการประมวลผล 2.4612744649251302e-05 นาที


# 40 % random nan

In [60]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'SMM40_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*40/100)) 
  index_nan = np.random.choice(globals()[f'SMM40_{i}']['SMM'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'SMM40_{i}']['SMM'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.17041015625 วินาที
เวลาที่ใช้ในการประมวลผล 0.0028401692708333332 นาที


## Imputation using Mean

In [61]:
from sklearn.impute import SimpleImputer

In [62]:
t_start = time.time()
mean40_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean40_{i}'] = mean40_imputer.fit_transform(globals()[f'SMM40_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN40_{i}'] = mean_squared_error(data, globals()[f'imputed_mean40_{i}']) # for loop mean_squared_error
t_end = time.time()
t40_MEAN = (t_end- t_start)

In [63]:
t_start = time.time()
mse_mean40_total = 0 
MSE_MEAN40_1001 = 0
for i in range(0,1001):
  mse_mean40_total = mse_mean40_total + globals()[f'MSE_MEAN40_{i+1}']
AMSE_SMM40_mean = mse_mean40_total/1000
print(f'AMSE_mean = {AMSE_SMM40_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mean = 0.2938419717653733
เวลาที่ใช้ในการประมวลผล 0.0011675357818603516 วินาที
เวลาที่ใช้ในการประมวลผล 1.9458929697672524e-05 นาที


## Imputation using the miss forest

In [64]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF40_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf40_{i}'] = MF40_imputer.fit_transform(globals()[f'SMM40_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF40_{i}'] = mean_squared_error(data, globals()[f'imputed_mf40_{i}']) # for loop mean_squared_error
t_end = time.time()
t40_MF = (t_end- t_start)

In [66]:
t_start = time.time()
mse_mf40_total = 0 
MSE_MF40_1001 = 0
for i in range(0,1001):
  mse_mf40_total = mse_mf40_total + globals()[f'MSE_MF40_{i+1}']
AMSE_SMM40_mf = mse_mf40_total/1000
print(f'AMSE_mf = {AMSE_SMM40_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mf = 0.03738432684137752
เวลาที่ใช้ในการประมวลผล 0.0015878677368164062 วินาที
เวลาที่ใช้ในการประมวลผล 2.6464462280273438e-05 นาที


## Imputation using the KNN

In [67]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [68]:
t_start = time.time()
KNN40_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN40_{i}'] = KNN40_imputer.fit_transform(globals()[f'SMM40_{i}'].copy())
  globals()[f'MSE_KNN40_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN40_{i}'])
t_end = time.time()
t40_KNN = (t_end- t_start)

In [69]:
t_start = time.time()
mse_knn40_total = 0 
MSE_KNN40_1001 = 0
for i in range(0,1001):
  mse_knn40_total = mse_knn40_total + globals()[f'MSE_KNN40_{i+1}']
AMSE_SMM40_knn = mse_knn40_total/1000
print(f'AMSE_knn = {AMSE_SMM40_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_knn = 0.2163293822021117
เวลาที่ใช้ในการประมวลผล 0.0009219646453857422 วินาที
เวลาที่ใช้ในการประมวลผล 1.5366077423095704e-05 นาที


## Imputation using the MICE

In [70]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr40_{i}'] = imp.fit_transform(globals()[f'SMM40_{i}'].copy())
  globals()[f'MSE_lr40_{i}'] = mean_squared_error(data, globals()[f'imputed_lr40_{i}'])
t_end = time.time()
t40_LR = (t_end- t_start)

In [72]:
t_start = time.time()
mse_lr40_total = 0 
MSE_lr40_1001 = 0
for i in range(0,1001):
  mse_lr40_total = mse_lr40_total + globals()[f'MSE_lr40_{i+1}']
AMSE_SMM40_lr = mse_lr40_total/1000
print(f'AMSE_lr = {AMSE_SMM40_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_lr = 0.05083116661799577
เวลาที่ใช้ในการประมวลผล 0.0016162395477294922 วินาที
เวลาที่ใช้ในการประมวลผล 2.6937325795491537e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm40_{i}'] = svm_svr.fit_transform(globals()[f'SMM40_{i}'].copy())
  globals()[f'MSE_svm40_{i}'] = mean_squared_error(data, globals()[f'imputed_svm40_{i}'])
t_end = time.time()
t40_SVM = (t_end- t_start)

In [74]:
t_start = time.time()
mse_svm40_total = 0 
MSE_svm40_1001 = 0
for i in range(0,1001):
  mse_svm40_total = mse_svm40_total + globals()[f'MSE_svm40_{i+1}']
AMSE_SMM40_svm = mse_svm40_total/1000
print(f'AMSE_svm = {AMSE_SMM40_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_svm = 0.25516776644768346
เวลาที่ใช้ในการประมวลผล 0.0013282299041748047 วินาที
เวลาที่ใช้ในการประมวลผล 2.2137165069580077e-05 นาที


## Imputation using the EM

### impy.em

In [75]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy40_{i}'] = impy.em(globals()[f'SMM40_{i}'].values.copy())
  globals()[f'MSE_impy40_{i}'] = mean_squared_error(data, globals()[f'imputed_impy40_{i}'])
t_end = time.time()
t40_EM = (t_end- t_start)

In [77]:
t_start = time.time()
mse_impy40_total = 0 
MSE_impy40_1001 = 0
for i in range(0,1001):
  mse_impy40_total = mse_impy40_total + globals()[f'MSE_impy40_{i+1}']
AMSE_SMM40_impy = mse_impy40_total/1000
print(f'AMSE_impy = {AMSE_SMM40_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_impy = 0.5570009824321538
เวลาที่ใช้ในการประมวลผล 0.0009801387786865234 วินาที
เวลาที่ใช้ในการประมวลผล 1.6335646311442057e-05 นาที


# 50 % random nan

In [78]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'SMM50_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*50/100)) 
  index_nan = np.random.choice(globals()[f'SMM50_{i}']['SMM'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'SMM50_{i}']['SMM'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.8406035900115967 วินาที
เวลาที่ใช้ในการประมวลผล 0.01401005983352661 นาที


## Imputation using Mean

In [79]:
from sklearn.impute import SimpleImputer

In [80]:
t_start = time.time()
mean50_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean50_{i}'] = mean50_imputer.fit_transform(globals()[f'SMM50_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN50_{i}'] = mean_squared_error(data, globals()[f'imputed_mean50_{i}']) # for loop mean_squared_error
t_end = time.time()
t50_MEAN = (t_end- t_start)

In [81]:
t_start = time.time()
mse_mean50_total = 0 
MSE_MEAN50_1001 = 0
for i in range(0,1001):
  mse_mean50_total = mse_mean50_total + globals()[f'MSE_MEAN50_{i+1}']
AMSE_SMM50_mean = mse_mean50_total/1000
print(f'AMSE_mean = {AMSE_SMM50_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mean = 0.36722146149623236
เวลาที่ใช้ในการประมวลผล 0.001027822494506836 วินาที
เวลาที่ใช้ในการประมวลผล 1.7130374908447264e-05 นาที


## Imputation using the miss forest

In [82]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF50_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf50_{i}'] = MF50_imputer.fit_transform(globals()[f'SMM50_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF50_{i}'] = mean_squared_error(data, globals()[f'imputed_mf50_{i}']) # for loop mean_squared_error
t_end = time.time()
t50_MF = (t_end- t_start)

In [84]:
t_start = time.time()
mse_mf50_total = 0 
MSE_MF50_1001 = 0
for i in range(0,1001):
  mse_mf50_total = mse_mf50_total + globals()[f'MSE_MF50_{i+1}']
AMSE_SMM50_mf = mse_mf50_total/1000
print(f'AMSE_mf = {AMSE_SMM50_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mf = 0.05121064234338861
เวลาที่ใช้ในการประมวลผล 0.0019516944885253906 วินาที
เวลาที่ใช้ในการประมวลผล 3.252824147542318e-05 นาที


## Imputation using the KNN

In [85]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [86]:
t_start = time.time()
KNN50_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN50_{i}'] = KNN50_imputer.fit_transform(globals()[f'SMM50_{i}'].copy())
  globals()[f'MSE_KNN50_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN50_{i}'])
t_end = time.time()
t50_KNN = (t_end- t_start)

In [87]:
t_start = time.time()
mse_knn50_total = 0 
MSE_KNN50_1001 = 0
for i in range(0,1001):
  mse_knn50_total = mse_knn50_total + globals()[f'MSE_KNN50_{i+1}']
AMSE_SMM50_knn = mse_knn50_total/1000
print(f'AMSE_knn = {AMSE_SMM50_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_knn = 0.2817176601418919
เวลาที่ใช้ในการประมวลผล 0.002415895462036133 วินาที
เวลาที่ใช้ในการประมวลผล 4.026492436726888e-05 นาที


## Imputation using the MICE

In [88]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr50_{i}'] = imp.fit_transform(globals()[f'SMM50_{i}'].copy())
  globals()[f'MSE_lr50_{i}'] = mean_squared_error(data, globals()[f'imputed_lr50_{i}'])
t_end = time.time()
t50_LR = (t_end- t_start)

In [90]:
t_start = time.time()
mse_lr50_total = 0 
MSE_lr50_1001 = 0
for i in range(0,1001):
  mse_lr50_total = mse_lr50_total + globals()[f'MSE_lr50_{i+1}']
AMSE_SMM50_lr = mse_lr50_total/1000
print(f'AMSE_lr = {AMSE_SMM50_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_lr = 0.07815360798627938
เวลาที่ใช้ในการประมวลผล 0.0012764930725097656 วินาที
เวลาที่ใช้ในการประมวลผล 2.1274884541829427e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm50_{i}'] = svm_svr.fit_transform(globals()[f'SMM50_{i}'].copy())
  globals()[f'MSE_svm50_{i}'] = mean_squared_error(data, globals()[f'imputed_svm50_{i}'])
t_end = time.time()
t50_SVM = (t_end- t_start)

In [92]:
t_start = time.time()
mse_svm50_total = 0 
MSE_svm50_1001 = 0
for i in range(0,1001):
  mse_svm50_total = mse_svm50_total + globals()[f'MSE_svm50_{i+1}']
AMSE_SMM50_svm = mse_svm50_total/1000
print(f'AMSE_svm = {AMSE_SMM50_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_svm = 0.33032709059676735
เวลาที่ใช้ในการประมวลผล 0.0014235973358154297 วินาที
เวลาที่ใช้ในการประมวลผล 2.3726622263590495e-05 นาที


## Imputation using the EM

### impy.em

In [93]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy50_{i}'] = impy.em(globals()[f'SMM50_{i}'].values.copy())
  globals()[f'MSE_impy50_{i}'] = mean_squared_error(data, globals()[f'imputed_impy50_{i}'])
t_end = time.time()
t50_EM = (t_end- t_start)

In [95]:
t_start = time.time()
mse_impy50_total = 0 
MSE_impy50_1001 = 0
for i in range(0,1001):
  mse_impy50_total = mse_impy50_total + globals()[f'MSE_impy50_{i+1}']
AMSE_SMM50_impy = mse_impy50_total/1000
print(f'AMSE_impy = {AMSE_SMM50_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_impy = 0.704255891492554
เวลาที่ใช้ในการประมวลผล 0.0009965896606445312 วินาที
เวลาที่ใช้ในการประมวลผล 1.6609827677408855e-05 นาที


# 60 % random nan

In [96]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'SMM60_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*60/100)) 
  index_nan = np.random.choice(globals()[f'SMM60_{i}']['SMM'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'SMM60_{i}']['SMM'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.15948724746704102 วินาที
เวลาที่ใช้ในการประมวลผล 0.0026581207911173504 นาที


## Imputation using Mean

In [97]:
from sklearn.impute import SimpleImputer

In [98]:
t_start = time.time()
mean60_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean60_{i}'] = mean50_imputer.fit_transform(globals()[f'SMM60_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN60_{i}'] = mean_squared_error(data, globals()[f'imputed_mean60_{i}']) # for loop mean_squared_error
t_end = time.time()
t60_MEAN = (t_end- t_start)

In [99]:
t_start = time.time()
mse_mean60_total = 0 
MSE_MEAN60_1001 = 0
for i in range(0,1001):
  mse_mean60_total = mse_mean60_total + globals()[f'MSE_MEAN60_{i+1}']
AMSE_SMM60_mean = mse_mean60_total/1000
print(f'AMSE_mean = {AMSE_SMM60_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mean = 0.44209146412579275
เวลาที่ใช้ในการประมวลผล 0.0011572837829589844 วินาที
เวลาที่ใช้ในการประมวลผล 1.9288063049316408e-05 นาที


## Imputation using the miss forest

In [100]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF60_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf60_{i}'] = MF60_imputer.fit_transform(globals()[f'SMM60_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF60_{i}'] = mean_squared_error(data, globals()[f'imputed_mf60_{i}']) # for loop mean_squared_error
t_end = time.time()
t60_MF = (t_end- t_start)

In [102]:
t_start = time.time()
mse_mf60_total = 0 
MSE_MF60_1001 = 0
for i in range(0,1001):
  mse_mf60_total = mse_mf60_total + globals()[f'MSE_MF60_{i+1}']
AMSE_SMM60_mf = mse_mf60_total/1000
print(f'AMSE_mf = {AMSE_SMM60_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_mf = 0.0704590108715435
เวลาที่ใช้ในการประมวลผล 0.0011138916015625 วินาที
เวลาที่ใช้ในการประมวลผล 1.8564860026041665e-05 นาที


## Imputation using the KNN

In [103]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [104]:
t_start = time.time()
KNN60_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN60_{i}'] = KNN60_imputer.fit_transform(globals()[f'SMM60_{i}'].copy())
  globals()[f'MSE_KNN60_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN60_{i}'])
t_end = time.time()
t60_KNN = (t_end- t_start)

In [105]:
t_start = time.time()
mse_knn60_total = 0 
MSE_KNN60_1001 = 0
for i in range(0,1001):
  mse_knn60_total = mse_knn60_total + globals()[f'MSE_KNN60_{i+1}']
AMSE_SMM60_knn = mse_knn60_total/1000
print(f'AMSE_knn = {AMSE_SMM60_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_knn = 0.354039690486565
เวลาที่ใช้ในการประมวลผล 0.006291627883911133 วินาที
เวลาที่ใช้ในการประมวลผล 0.00010486046473185221 นาที


## Imputation using the MICE

In [106]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr60_{i}'] = imp.fit_transform(globals()[f'SMM60_{i}'].copy())
  globals()[f'MSE_lr60_{i}'] = mean_squared_error(data, globals()[f'imputed_lr60_{i}'])
t_end = time.time()
t60_LR = (t_end- t_start)

In [108]:
t_start = time.time()
mse_lr60_total = 0 
MSE_lr60_1001 = 0
for i in range(0,1001):
  mse_lr60_total = mse_lr60_total + globals()[f'MSE_lr60_{i+1}']
AMSE_SMM60_lr = mse_lr60_total/1000
print(f'AMSE_lr = {AMSE_SMM60_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_lr = 0.11153017600168427
เวลาที่ใช้ในการประมวลผล 0.0009639263153076172 วินาที
เวลาที่ใช้ในการประมวลผล 1.6065438588460288e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm60_{i}'] = svm_svr.fit_transform(globals()[f'SMM60_{i}'].copy())
  globals()[f'MSE_svm60_{i}'] = mean_squared_error(data, globals()[f'imputed_svm60_{i}'])
t_end = time.time()
t60_SVM = (t_end- t_start)

In [110]:
t_start = time.time()
mse_svm60_total = 0 
MSE_svm60_1001 = 0
for i in range(0,1001):
  mse_svm60_total = mse_svm60_total + globals()[f'MSE_svm60_{i+1}']
AMSE_SMM60_svm = mse_svm60_total/1000
print(f'AMSE_svm = {AMSE_SMM60_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_svm = 0.41171085619992875
เวลาที่ใช้ในการประมวลผล 0.0012729167938232422 วินาที
เวลาที่ใช้ในการประมวลผล 2.1215279897054037e-05 นาที


## Imputation using the EM

### impy.em

In [111]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [112]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy60_{i}'] = impy.em(globals()[f'SMM60_{i}'].values.copy())
  globals()[f'MSE_impy60_{i}'] = mean_squared_error(data, globals()[f'imputed_impy60_{i}'])
t_end = time.time()
t60_EM = (t_end- t_start)

In [113]:
t_start = time.time()
mse_impy60_total = 0 
MSE_impy60_1001 = 0
for i in range(0,1001):
  mse_impy60_total = mse_impy60_total + globals()[f'MSE_impy60_{i+1}']
AMSE_SMM60_impy = mse_impy60_total/1000
print(f'AMSE_impy = {AMSE_SMM60_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_impy = 0.8723297805245731
เวลาที่ใช้ในการประมวลผล 0.0009579658508300781 วินาที
เวลาที่ใช้ในการประมวลผล 1.5966097513834636e-05 นาที


# AMSE

In [116]:
amse_value = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[AMSE_SMM10_mean,AMSE_SMM10_mf,AMSE_SMM10_knn,AMSE_SMM10_lr,AMSE_SMM10_svm,AMSE_SMM10_impy],
      '20%':[AMSE_SMM20_mean,AMSE_SMM20_mf,AMSE_SMM20_knn,AMSE_SMM20_lr,AMSE_SMM20_svm,AMSE_SMM20_impy],
      '30%':[AMSE_SMM30_mean,AMSE_SMM30_mf,AMSE_SMM30_knn,AMSE_SMM30_lr,AMSE_SMM30_svm,AMSE_SMM30_impy],
      '40%':[AMSE_SMM40_mean,AMSE_SMM40_mf,AMSE_SMM40_knn,AMSE_SMM40_lr,AMSE_SMM40_svm,AMSE_SMM40_impy],
      '50%':[AMSE_SMM50_mean,AMSE_SMM50_mf,AMSE_SMM50_knn,AMSE_SMM50_lr,AMSE_SMM60_svm,AMSE_SMM50_impy]
      ,'60%':[AMSE_SMM60_mean,AMSE_SMM60_mf,AMSE_SMM60_knn,AMSE_SMM60_lr,AMSE_SMM60_svm,AMSE_SMM60_impy]}

In [117]:
data_AMSE = pd.DataFrame.from_dict(amse_value)
data_AMSE

,Method,10%,20%,30%,40%,50%,60%
0,MEAN,0.073696,0.145715,0.218100,0.293842,0.367221,0.442091
1,Miss Forest,0.008378,0.016926,0.026321,0.037384,0.051211,0.070459
2,KNN,0.050255,0.101978,0.155894,0.216329,0.281718,0.354040
3,MICE-LR,0.006640,0.016610,0.033353,0.050831,0.078154,0.111530
4,MICE-SVM,0.059067,0.119567,0.183674,0.255168,0.411711,0.411711
5,EM,0.137714,0.270052,0.411161,0.557001,0.704256,0.872330


# เวลาทั้งหมดในการรัน

## second time

In [118]:
data_t_sec = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[t10_MEAN,t10_MF,t10_KNN,t10_LR,t10_SVM,t10_EM],
      '20%':[t20_MEAN,t20_MF,t20_KNN,t20_LR,t20_SVM,t20_EM],'30%':[t30_MEAN,t30_MF,t30_KNN,t30_LR,t30_SVM,t30_EM],'40%':[t40_MEAN,t40_MF,t40_KNN,t40_LR,t40_SVM,t40_EM],
      '50%':[t50_MEAN,t50_MF,t50_KNN,t50_LR,t50_SVM,t50_EM],'60%':[t60_MEAN,t60_MF,t60_KNN,t60_LR,t60_SVM,t60_EM]}

In [119]:
data_second = pd.DataFrame.from_dict(data_t_sec)
data_second

,Method,10%,20%,30%,40%,50%,60%
0,MEAN,3.836174,3.680744,3.900198,3.919160,3.844975,4.127775
1,Miss Forest,702.493090,674.777505,683.457673,670.718507,681.793504,672.849068
2,KNN,4.566862,4.678110,4.923002,4.993066,8.932688,9.632754
3,MICE-LR,50.443089,50.402169,53.406913,52.078038,53.176040,52.686897
4,MICE-SVM,111.843728,112.003518,112.640847,113.109482,115.241671,115.411843
5,EM,6.136305,12.152719,18.267288,23.891612,28.493966,33.622127


## minutes

In [120]:
data_t_min = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[t10_MEAN/60,t10_MF/60,t10_KNN/60,t10_LR/60,t10_SVM/60,t10_EM/60],
      '20%':[t20_MEAN/60,t20_MF/60,t20_KNN/60,t20_LR/60,t20_SVM/60,t20_EM/60],'30%':[t30_MEAN/60,t30_MF/60,t30_KNN/60,t30_LR/60,t30_SVM/60,t30_EM/60]
      ,'40%':[t40_MEAN/60,t40_MF/60,t40_KNN/60,t40_LR/60,t40_SVM/60,t40_EM/60],'50%':[t50_MEAN/60,t50_MF/60,t50_KNN/60,t50_LR/60,t50_SVM/60,t50_EM/60],
      '60%':[t60_MEAN/60,t60_MF/60,t60_KNN/60,t60_LR/60,t60_SVM/60,t60_EM/60]}
     

In [121]:
data_minutes = pd.DataFrame.from_dict(data_t_min)
data_minutes
     

,Method,10%,20%,30%,40%,50%,60%
0,MEAN,0.063936,0.061346,0.065003,0.065319,0.064083,0.068796
1,Miss Forest,11.708218,11.246292,11.390961,11.178642,11.363225,11.214151
2,KNN,0.076114,0.077969,0.082050,0.083218,0.148878,0.160546
3,MICE-LR,0.840718,0.840036,0.890115,0.867967,0.886267,0.878115
4,MICE-SVM,1.864062,1.866725,1.877347,1.885158,1.920695,1.923531
5,EM,0.102272,0.202545,0.304455,0.398194,0.474899,0.560369
